Answerable keywords:
- yes: question is answerable given the dpr answer
- no: question is not answerable given the dpr answer
- check: question would have been answerable given better sentence splits
- interesting: dpr answer is vague about the answer
- invalid: question is invalid because of incorrect first name of second qnode
- incorrect: dpr answer has wrong information (wikipedia obsolete/incorrect)

In [84]:
import json
import random
import pandas as pd
from utils import match_dates_based_on_precision

/nas/home/amandeep/miniconda3/envs/python3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [71]:
input_file = '../data/dpr-post-process-unified-qa-matched.jl'
metrics_file = '../data/unified-qa-metrics.tsv'
random_output_file = '../data/questions-for-annotation.csv'

In [72]:
q_to_p_dict = json.load(open('../data/questions_o_precision_dict.json'))

In [73]:
precision_dict = {
    
    "14": "second",
    "13": "minute",
    "12": "hour",
    "11": "day",
    "10": "month",
    "9": "year",
    "8": "decade",
    "7": "century",
    "6": "millenium",
    "4": "hundred thousand years",
    "3": "million years",
    "0": "billion years"
}

In [35]:
o = open(metrics_file, 'w')

In [36]:
o.write('question\tanswer\tprecision\tdpr_para\trank\tfiltered_rank\trag_matched\tunified_qa_matched\tunified_qa_answer\tagreement\n')

113

In [37]:
with open(input_file) as f:
    for line in f:
        x = json.loads(line.strip())
        question = x['q']
        answer = x['a']
        precision = precision_dict[q_to_p_dict[question]]
        mda = x['matched_dpr_answers']
        for i, da in enumerate(mda):
            dpr_answer = da['dpr_answer'].replace('\n', ' ')
            u_a = da['unified_qa_answers']
            if len(u_a) > 0:
                u_a = u_a[0]
            else:
                u_a = ''
            
            o.write(f"{question}\t{answer}\t{precision}\t{dpr_answer}\t \
            {da['rank']}\t{i+1}\t{da['matched']}\t{da['uqa_matched']}\t{u_a}\t{da['matched'] == da['uqa_matched']}\n")

In [38]:
o.close()

In [42]:
year_matched = 0
month_matched = 0
exact_match = 0
precision_matched = 0
total = 0
with open(input_file) as f:
    for line in f:
        total += 1
        x = json.loads(line.strip())
        question = x['q']
        answer = x['a']
        year, month, day = answer.split('-')
        mda = x['matched_dpr_answers']
        for i, da in enumerate(mda):
            if da['uqa_matched']:
                precision_matched += 1
            u_a = da['unified_parsed_dates']
            if len(u_a) > 0:
                u_a = u_a[0]
                if answer == u_a['date']:
                    exact_match += 1
                if year == str(u_a['year']):
                    year_matched += 1
                if month == u_a['month']:
                    month_matched += 1

print(total)
print(precision_matched)
print(year_matched)
print(month_matched)
print(exact_match)

13382
12619
13087
5501
5219


In [91]:
def create_random_questions(input_file, output_file, k=100):
    all_questions = []
    out = []
    with open(input_file) as f:
        for line in f:
            x = json.loads(line.strip())
            all_questions.append(x)
    random_k = random.sample(all_questions, k)
    for x in random_k:
        question = x['q']
        answer = x['a']
        precision = precision_dict[q_to_p_dict[question]]
        mda = x['matched_dpr_answers']            
            
        # get the highest ranked dpr answer only, which is the first one because of sorting beforehand
        mda_best = mda[0]
        parsed_dates = mda_best['parsed_dates']
        non_matched_dates = set()
        for parsed_d in parsed_dates:
            matched, _ =  match_dates_based_on_precision(answer, precision, parsed_d)
            if not matched:
                non_matched_dates.add(parsed_d['date'])
        dpr_answer = mda_best['dpr_answer'].replace('\n', ' ').replace('"', '')
        out.append({
            'question': question,
            'answer': answer,
            'precision': precision,
            'dpr_answer': dpr_answer,
            'rank': mda_best['rank'],
            'filtered_rank': i+1,
            'other_dates': ",".join(non_matched_dates)
        })
    pd.DataFrame(out).to_csv(output_file, index=False)

          

In [92]:
create_random_questions(input_file, random_output_file, k=120)